<a href="https://colab.research.google.com/github/tamagoyakitamagoyaki/q-macro/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

In [23]:
data = pwt1001.loc[pwt1001['country'].isin(['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States'])][['year', 'countrycode', 'country', 'rgdpna', 'rnna', 'emp', 'avh', 'labsh', 'hc', 'rtfpna']]
data = data.loc[(data['year'] >= 1975) & (data['year'] <= 2019)].dropna()

In [24]:
data['alpha'] = 1 - data['labsh']
data['gdp_per_worker'] = data['rgdpna'] / data['emp']
data['capital_per_worker'] = data['rnna'] / data['emp']
data['human_capital_per_worker'] = data['hc']
data['hours_per_worker'] = data['avh']

In [25]:
countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']

In [28]:
def calculate_average_growth_rate(data, start_year, end_year, column):
    start_value = data.loc[data['year'] == start_year, column].iloc[0]
    end_value = data.loc[data['year'] == end_year, column].iloc[0]
    years = end_value - start_value
    avg_growth_rate = (end_value / start_value) ** (1 / years) - 1
    return avg_growth_rate

In [34]:
results = []
for country in countries:
    country_data = data[data['country'] == country]
    alpha = country_data['alpha'].mean()
    gdp_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'gdp_per_worker')
    capital_deepening = calculate_average_growth_rate(country_data, 1975, 2019, 'capital_per_worker')
    hc_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'human_capital_per_worker')
    hours_growth = calculate_average_growth_rate(country_data, 1975, 2019, 'hours_per_worker')
    labor_growth = (1 - alpha) * (hc_growth + hours_growth)
    tfp_growth = gdp_growth - capital_deepening - labor_growth
    tfp_share = tfp_growth / gdp_growth
    capital_share = capital_deepening / gdp_growth
    labor_share = labor_growth / gdp_growth
    results.append({
        'Country': country,
        'Growth Rate': gdp_growth,
        'TFP Growth': tfp_growth,
        'Capital Deepening': capital_deepening,
        'Labor Growth': labor_growth,
        'TFP Share': tfp_share,
        'Capital Share': capital_share,
        'Labor Share': labor_share
     })


In [37]:
growth_accounting_table = pd.DataFrame(results)

average_values = {
    'Country': 'Average',
    'Growth Rate': growth_accounting_table['Growth Rate'].mean(),
    'TFP Growth': growth_accounting_table['TFP Growth'].mean(),
    'Capital Deepening': growth_accounting_table['Capital Deepening'].mean(),
    'Labor Growth': growth_accounting_table['Labor Growth'].mean(),
    'TFP Share': growth_accounting_table['TFP Share'].mean(),
    'Capital Share': growth_accounting_table['Capital Share'].mean(),
    'Labor Share': growth_accounting_table['Labor Share'].mean()
}

In [40]:
average_value_df = pd.DataFrame([average_values])
growth_accounting_table = pd.concat([growth_accounting_table,average_value_df], ignore_index=True)
print(growth_accounting_table.to_string(index=False, float_format='{:.2%}'.format))

       Country  Growth Rate  TFP Growth  Capital Deepening  Labor Growth    TFP Share  Capital Share  Labor Share
     Australia        0.00%     -21.77%              0.00%        21.78% -1735713.67%         22.10%  1735791.57%
       Austria        0.00%     -23.74%              0.00%        23.74% -1840633.16%         20.05%  1840713.11%
       Belgium        0.00%     -27.05%              0.00%        27.05% -2200272.21%         18.84%  2200353.37%
        Canada        0.00%     -23.70%              0.00%        23.70% -1890850.33%         25.64%  1890924.69%
       Denmark        0.00%     -23.38%              0.00%        23.39% -1795661.79%         21.78%  1795740.01%
       Finland        0.00%     -24.37%              0.00%        24.37% -1539997.81%         20.86%  1540076.95%
        France        0.00%     -27.04%              0.00%        27.04% -2117121.18%         18.38%  2117202.80%
       Germany        0.00%     -22.57%              0.00%        22.57% -1603597.74%   